# 🚀 BTC 5m LSTM Trading - Google Colab

## การเทรน LSTM Model สำหรับเทรด Bitcoin 5 นาที

### Features:
- ✅ Walk-Forward Analysis
- ✅ Triple Barrier Labeling
- ✅ Risk-Reward Ratio 1:2
- ✅ Technical Indicators (RSI, EMA, ATR)
- ✅ LSTM Classifier

---

## 🔧 Setup และติดตั้ง

### 1. Clone Repository และติดตั้ง Dependencies

In [15]:
# Clone repository
!git clone https://github.com/tey333/btc5m-lstm-starter.git
%cd btc5m-lstm-starter

Cloning into 'btc5m-lstm-starter'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 88 (delta 23), reused 81 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (88/88), 13.61 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/btc5m-lstm-starter/btc5m-lstm-starter


In [2]:
# รัน setup script
!python colab_setup.py

🚀 Setting up BTC 5m LSTM Trading for Google Colab
📁 Setting up directories...
✅ Created: data/prepared
✅ Created: data/raw
✅ Created: data/sample
✅ Created: outputs/models
✅ Created: outputs/trades
✅ Created: outputs/metrics
🔧 Installing required packages...
Installing numpy...
Installing pandas...
Installing pyarrow...
Installing torch...
Installing scikit-learn...
Installing ta...
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=b89a728db206d8412e1572ab95a286402ce246eacdefda369d00376b583f3daf
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
Installing tqdm...
Installing pyyaml...
🎯 Checking GPU availability...
✅ GPU available: NVIDIA L4
GPU Memory: 22.2 GB
💡 แนะนำ batch_size: 1024
📊 Checking for sample data...
✅ Sample data found: data/sample/btc5m_sample.csv

✅ Setup completed!

Next steps:
1. Upload your BTC 5m data to 'data/raw/' fol

In [3]:
# สร้างไฟล์ calculate_windows.py สำหรับ Colab
calculate_windows_code = '''#!/usr/bin/env python3
"""
คำนวณจำนวน Windows ที่จะเทรนจาก Configuration
ใช้สำหรับการคำนวณ Walk-Forward Analysis windows
"""

import yaml
import pandas as pd
from pathlib import Path
import sys
import os

def calculate_windows_simple(config_path="configs/config.yaml"):
    """คำนวณจำนวน windows จาก config แบบง่าย"""

    # โหลด config
    with open(config_path, 'r', encoding='utf-8') as f:
        cfg = yaml.safe_load(f)

    print("🔧 Configuration:")
    print(f"  📅 Train months: {cfg['split']['train_months']}")
    print(f"  ✅ Valid months: {cfg['split']['valid_months']}")
    print(f"  🧪 Test months: {cfg['split']['test_months']}")
    print(f"  ➡️ Step months: {cfg['split']['step_months']}")
    print()

    # ตรวจสอบไฟล์ข้อมูล
    data_path = None
    prepared_dir = Path("data/prepared")
    if prepared_dir.exists():
        data_files = list(prepared_dir.glob("*.parquet"))
        if data_files:
            data_path = data_files[0]

    if data_path is None:
        print("❌ ไม่พบไฟล์ข้อมูลที่เตรียมแล้ว!")
        print("💡 กรุณารันการ preprocessing ก่อน")
        return None

    print(f"📊 กำลังโหลดข้อมูลจาก: {data_path}")

    # โหลดข้อมูล
    df = pd.read_parquet(data_path)
    print(f"📈 ข้อมูลทั้งหมด: {len(df):,} รายการ")

    # แปลง timestamp
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    start_date = df['timestamp'].min()
    end_date = df['timestamp'].max()

    print(f"📅 ระยะเวลาข้อมูล: {start_date.strftime('%Y-%m-%d')} ถึง {end_date.strftime('%Y-%m-%d')}")
    total_days = (end_date - start_date).days
    total_months = total_days / 30.44
    print(f"⏱️ รวม: {total_days} วัน ({total_months:.1f} เดือน)")
    print()

    # คำนวณ windows โดยประมาณ
    window_size = cfg['split']['train_months'] + cfg['split']['valid_months'] + cfg['split']['test_months']
    step_size = cfg['split']['step_months']

    # จำนวน windows = (total_months - window_size) / step_size + 1
    num_windows = max(0, int((total_months - window_size) / step_size) + 1)

    # แสดงสรุป
    print("=" * 60)
    print(f"🎯 สรุปผลการคำนวณ:")
    print(f"  📊 Window size: {window_size} เดือน")
    print(f"  ➡️ Step size: {step_size} เดือน")
    print(f"  ✅ จำนวน Windows: {num_windows}")
    print()

    if num_windows > 0:
        print(f"🚀 การเทรนจะแสดง: [Window 0] ถึง [Window {num_windows-1}]")
        print(f"⏱️ คาดว่าใช้เวลา: {num_windows * 10:.0f}-{num_windows * 25:.0f} นาที")
        print(f"   (ประมาณ 10-25 นาทีต่อ Window ขึ้นอยู่กับ GPU)")

        # คำนวณ breakeven win rate
        with open(config_path, 'r', encoding='utf-8') as f:
            config = yaml.safe_load(f)

        rr_ratio = config['trade']['atr_mult_tp'] / config['trade']['atr_mult_sl']
        breakeven_wr = 1 / (1 + rr_ratio)
        print(f"📊 Breakeven Win Rate: {breakeven_wr:.1%}")

    else:
        print("❌ ไม่มี Window ที่ใช้งานได้!")
        print("💡 ลองปรับลด train_months, valid_months, test_months")

    print("=" * 60)

    return num_windows

if __name__ == "__main__":
    calculate_windows_simple()
'''

# เขียนไฟล์
with open('calculate_windows.py', 'w', encoding='utf-8') as f:
    f.write(calculate_windows_code)

print("✅ สร้างไฟล์ calculate_windows.py เรียบร้อย!")

✅ สร้างไฟล์ calculate_windows.py เรียบร้อย!


## 📊 Upload และเตรียมข้อมูล

### 2. Upload ไฟล์ข้อมูล BTC 5m

In [4]:
from google.colab import files
import os

# Upload ไฟล์ข้อมูล
print("🔼 Upload ไฟล์ข้อมูล BTC 5m (.csv หรือ .parquet)")
print("Format ที่ต้องการ: timestamp, open, high, low, close, volume")

uploaded = files.upload()

# ย้ายไฟล์ไปยัง data/raw/
for filename in uploaded.keys():
    os.rename(filename, f'data/raw/{filename}')
    print(f"✅ ย้ายไฟล์ {filename} ไปยัง data/raw/")

🔼 Upload ไฟล์ข้อมูล BTC 5m (.csv หรือ .parquet)
Format ที่ต้องการ: timestamp, open, high, low, close, volume


Saving raw.csv to raw.csv
✅ ย้ายไฟล์ raw.csv ไปยัง data/raw/


### 3. Preprocessing ข้อมูล

In [5]:
# รัน preprocessing
!python -m scripts.preprocess

Saved: data/prepared/btc_5m_clean.parquet | shape: (792661, 6) | range: 2018-02-15 17:00:00+00:00 → 2025-08-30 00:00:00+00:00


## ⚙️ ตรวจสอบและปรับ Configuration

### 4. ดู Config ปัจจุบัน

In [6]:
# แสดง config ปัจจุบัน
!cat configs/config.yaml

# ================================
# BTC 5m LSTM Trading Configuration
# สำหรับการเทรนและใช้งานจริง
# ================================
#
# การใช้งาน:
# - สำหรับการทดสอบ: ลด train_months, max_epochs, batch_size
# - สำหรับการเทรนจริง: ใช้ค่าปัจจุบัน
# - สำหรับ CPU: เปลี่ยน device เป็น "cpu" และลด batch_size
#
# ================================
# การตั้งค่าข้อมูล (Data Configuration)
# ================================
data:
  raw_glob: "data/raw/*.*"     # ไฟล์ข้อมูลดิบ รองรับ .csv หรือ .parquet
  tz_source: "Asia/Bangkok"    # timezone ของข้อมูลต้นฉบับ (เปลี่ยนตามต้องการ)
  tz_target: "UTC"             # แปลงเป็น UTC สำหรับการประมวลผลภายใน
  ensure_regular_5m: true      # เติมช่องว่างให้ข้อมูลเป็น 5 นาทีสม่ำเสมอ

# ================================
# การสร้างฟีเจอร์ (Feature Engineering)
# ================================
feature:
  rsi_period: 14                         # ช่วงเวลาการคำนวณ RSI (Relative Strength Index)
  ema_periods: [8, 21, 55, 144]          # EMA Fibonacci sequence: เหม

### 5. ตรวจสอบ GPU และปรับ Config

### 5. ปรับแต่ง Config เพื่อผลลัพธ์ที่ดีกว่า

เลือก strategy ที่เหมาะสม:

In [7]:
# 🎯 เลือก Strategy สำหรับผลลัพธ์ที่ดีกว่า
import yaml

print("🔧 เลือก Trading Strategy:")
print("1️⃣ Conservative (Win Rate สูง, RR 1:1.2)")
print("2️⃣ Balanced (Win Rate ปานกลาง, RR 1:1.5)")
print("3️⃣ Aggressive (Win Rate ต่ำ, RR 1:2)")
print("4️⃣ Scalping (Win Rate สูงมาก, RR 1:1)")

strategy = input("\nเลือก strategy (1-4): ").strip()

# Load current config
with open('configs/config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

if strategy == "1":
    print("📈 ใช้ Conservative Strategy...")
    # Conservative: เน้น Win Rate สูง
    config['label']['atr_mult_tp'] = 1.2
    config['label']['atr_mult_sl'] = 1.0
    config['label']['max_holding'] = 36
    config['trade']['atr_mult_tp'] = 1.2
    config['trade']['atr_mult_sl'] = 1.0
    config['trade']['max_holding'] = 36
    config['trade']['proba_threshold'] = 0.70
    config['train']['class_weights'] = [1.8, 0.2, 1.8]
    config['split']['train_months'] = 8
    config['split']['step_months'] = 2

elif strategy == "2":
    print("⚖️ ใช้ Balanced Strategy...")
    # Balanced: สมดุลระหว่าง Win Rate และ RR
    config['label']['atr_mult_tp'] = 1.5
    config['label']['atr_mult_sl'] = 1.0
    config['label']['max_holding'] = 24
    config['trade']['atr_mult_tp'] = 1.5
    config['trade']['atr_mult_sl'] = 1.0
    config['trade']['max_holding'] = 24
    config['trade']['proba_threshold'] = 0.60
    config['train']['class_weights'] = [1.5, 0.3, 1.5]
    config['split']['train_months'] = 6
    config['split']['step_months'] = 1

elif strategy == "3":
    print("🚀 ใช้ Aggressive Strategy...")
    # Aggressive: เน้น RR สูง
    config['label']['atr_mult_tp'] = 2.0
    config['label']['atr_mult_sl'] = 1.0
    config['label']['max_holding'] = 48
    config['trade']['atr_mult_tp'] = 2.0
    config['trade']['atr_mult_sl'] = 1.0
    config['trade']['max_holding'] = 48
    config['trade']['proba_threshold'] = 0.55
    config['train']['class_weights'] = [1.2, 0.5, 1.2]
    config['split']['train_months'] = 12
    config['split']['step_months'] = 3

elif strategy == "4":
    print("⚡ ใช้ Scalping Strategy...")
    # Scalping: เน้น trades มาก Win Rate สูง
    config['label']['atr_mult_tp'] = 1.0
    config['label']['atr_mult_sl'] = 1.0
    config['label']['max_holding'] = 12
    config['trade']['atr_mult_tp'] = 1.0
    config['trade']['atr_mult_sl'] = 1.0
    config['trade']['max_holding'] = 12
    config['trade']['proba_threshold'] = 0.45
    config['train']['class_weights'] = [2.5, 0.1, 2.5]
    config['split']['train_months'] = 4
    config['split']['step_months'] = 1
    config['train']['seq_len'] = 24
else:
    print("❌ เลือกไม่ถูกต้อง ใช้ค่าเดิม")
    strategy = None

if strategy in ["1", "2", "3", "4"]:
    # Save updated config
    with open('configs/config.yaml', 'w', encoding='utf-8') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

    print("✅ อัปเดต config เรียบร้อย!")
    print(f"📊 RR: {config['trade']['atr_mult_tp']:.1f}:{config['trade']['atr_mult_sl']:.1f}")
    print(f"⏰ Max holding: {config['trade']['max_holding']} periods")
    print(f"🎯 Probability threshold: {config['trade']['proba_threshold']}")
    print(f"📈 Train months: {config['split']['train_months']}")
else:
    print("⏭️ ข้ามการอัปเดต config")

🔧 เลือก Trading Strategy:
1️⃣ Conservative (Win Rate สูง, RR 1:1.2)
2️⃣ Balanced (Win Rate ปานกลาง, RR 1:1.5)
3️⃣ Aggressive (Win Rate ต่ำ, RR 1:2)
4️⃣ Scalping (Win Rate สูงมาก, RR 1:1)

เลือก strategy (1-4): 3
🚀 ใช้ Aggressive Strategy...
✅ อัปเดต config เรียบร้อย!
📊 RR: 2.0:1.0
⏰ Max holding: 48 periods
🎯 Probability threshold: 0.55
📈 Train months: 12


### 6. ตรวจสอบการตั้งค่าที่ปรับแล้ว

In [8]:
# ตรวจสอบ config ที่ปรับแล้ว และคำนวณ windows
!python calculate_windows.py

print("\n" + "="*50)
print("📊 สรุปการตั้งค่าปัจจุบัน:")

with open('configs/config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# แสดงการตั้งค่าสำคัญ
print(f"🎯 Strategy: RR {config['trade']['atr_mult_tp']:.1f}:{config['trade']['atr_mult_sl']:.1f}")
print(f"⏰ Max Holding: {config['trade']['max_holding']} periods ({config['trade']['max_holding']*5:.0f} นาที)")
print(f"📈 Probability Threshold: {config['trade']['proba_threshold']}")
print(f"🧠 Training Period: {config['split']['train_months']} เดือน")
print(f"➡️ Step Size: {config['split']['step_months']} เดือน")
print(f"📏 Sequence Length: {config['train']['seq_len']} periods ({config['train']['seq_len']*5:.0f} นาที)")
print(f"⚖️ Class Weights: {config['train']['class_weights']}")

# คำนวณ breakeven win rate
rr_ratio = config['trade']['atr_mult_tp'] / config['trade']['atr_mult_sl']
breakeven_wr = 1 / (1 + rr_ratio)
print(f"📊 Breakeven Win Rate: {breakeven_wr:.1%} (ต้องชนะมากกว่านี้เพื่อกำไร)")

print("="*50)

🔧 Configuration:
  📅 Train months: 12
  ✅ Valid months: 1
  🧪 Test months: 1
  ➡️ Step months: 3

📊 กำลังโหลดข้อมูลจาก: data/prepared/btc_5m_clean.parquet
📈 ข้อมูลทั้งหมด: 792,661 รายการ
📅 ระยะเวลาข้อมูล: 2018-02-15 ถึง 2025-08-30
⏱️ รวม: 2752 วัน (90.4 เดือน)

🎯 สรุปผลการคำนวณ:
  📊 Window size: 14 เดือน
  ➡️ Step size: 3 เดือน
  ✅ จำนวน Windows: 26

🚀 การเทรนจะแสดง: [Window 0] ถึง [Window 25]
⏱️ คาดว่าใช้เวลา: 260-650 นาที
   (ประมาณ 10-25 นาทีต่อ Window ขึ้นอยู่กับ GPU)
📊 Breakeven Win Rate: 33.3%

📊 สรุปการตั้งค่าปัจจุบัน:
🎯 Strategy: RR 2.0:1.0
⏰ Max Holding: 48 periods (240 นาที)
📈 Probability Threshold: 0.55
🧠 Training Period: 12 เดือน
➡️ Step Size: 3 เดือน
📏 Sequence Length: 144 periods (720 นาที)
⚖️ Class Weights: [1.2, 0.5, 1.2]
📊 Breakeven Win Rate: 33.3% (ต้องชนะมากกว่านี้เพื่อกำไร)


In [9]:
import torch
import yaml

# ตรวจสอบ GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU: {gpu_name}")
    print(f"📱 Memory: {gpu_memory:.1f} GB")

    # แนะนำ batch_size ตาม GPU memory
    if gpu_memory < 8:
        recommended_batch = 256
    elif gpu_memory < 16:
        recommended_batch = 512
    else:
        recommended_batch = 1024

    print(f"💡 แนะนำ batch_size: {recommended_batch}")

else:
    print("⚠️ ไม่มี GPU, จะใช้ CPU")
    print("💡 แนะนำ: batch_size: 256, max_epochs: 5-10")

✅ GPU: NVIDIA L4
📱 Memory: 22.2 GB
💡 แนะนำ batch_size: 1024


## 🎯 เริ่มการเทรนแบบ Optimized

### 7. เทรน Model พร้อม Progress Monitoring

In [10]:
# เริ่มการเทรนพร้อม monitoring
import time
start_time = time.time()

print("🚀 เริ่มการเทรน LSTM Model...")
print("💡 Tips: หากต้องการหยุดกะทันหัน ใช้ Runtime > Interrupt execution")
print("📊 Progress จะแสดงแต่ละ Window ที่เทรน\n")

# รันการเทรน
!python -m scripts.run_train

# คำนวณเวลาที่ใช้
end_time = time.time()
duration = (end_time - start_time) / 60
print(f"\n⏱️ เทรนเสร็จแล้ว! ใช้เวลา: {duration:.1f} นาที")

# ตรวจสอบผลลัพธ์เบื้องต้น
print("\n🔍 ตรวจสอบผลลัพธ์เบื้องต้น...")
!ls -la outputs/

🚀 เริ่มการเทรน LSTM Model...
💡 Tips: หากต้องการหยุดกะทันหัน ใช้ Runtime > Interrupt execution
📊 Progress จะแสดงแต่ละ Window ที่เทรน

[Window 0] train=105120 valid=8064 test=8928
[Window 1] train=105120 valid=8928 test=8640
[Window 2] train=105120 valid=8928 test=8640
[Window 3] train=105120 valid=8640 test=8928
[Window 4] train=105120 valid=8352 test=8928
[Window 5] train=105408 valid=8928 test=8640
[Window 6] train=105408 valid=8928 test=8640
[Window 7] train=105408 valid=8640 test=8928
[Window 8] train=105408 valid=8064 test=8928
[Window 9] train=105120 valid=8928 test=8640
[Window 10] train=105120 valid=8928 test=8640
[Window 11] train=105120 valid=8640 test=8928
[Window 12] train=105120 valid=8064 test=8928
[Window 13] train=105120 valid=8928 test=8640
[Window 14] train=105120 valid=8928 test=8640
[Window 15] train=105120 valid=8640 test=8928
[Window 16] train=105120 valid=8064 test=8928
[Window 17] train=105120 valid=8928 test=8640
[Window 18] train=105120 valid=8928 test=8640
[Wi

## 📈 ตรวจสอบผลลัพธ์

### 7. ดูผลการเทรน

In [11]:
import pandas as pd
import os

# ตรวจสอบไฟล์ผลลัพธ์
results_dir = 'outputs'

print("📁 ไฟล์ในโฟลเดอร์ outputs:")
for root, dirs, files in os.walk(results_dir):
    level = root.replace(results_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

📁 ไฟล์ในโฟลเดอร์ outputs:
outputs/
  trades/
    trades_w5.parquet
    trades_w7.parquet
    trades_w3.parquet
    trades_w1.parquet
    trades_w9.parquet
    trades_w12.parquet
    trades_w2.parquet
    trades_w8.parquet
    trades_w18.parquet
    trades_w20.parquet
    trades_w6.parquet
    trades_w22.parquet
    trades_w0.parquet
    trades_w13.parquet
    trades_w21.parquet
    trades_w17.parquet
    trades_w23.parquet
    trades_w4.parquet
    trades_w24.parquet
    trades_w19.parquet
    trades_w14.parquet
    trades_w25.parquet
    trades_w10.parquet
    .gitkeep
    trades_w15.parquet
    trades_w11.parquet
    trades_w16.parquet
  metrics/
    metrics_windows.json
    summary.json
    .gitkeep
    windows_summary.csv
  models/
    scaler_std_w7.npy
    scaler_std_w3.npy
    scaler_mean_w22.npy
    lstm_w11.pt
    lstm_w8.pt
    scaler_mean_w4.npy
    scaler_mean_w17.npy
    scaler_mean_w5.npy
    scaler_std_w1.npy
    scaler_mean_w18.npy
    scaler_std_w21.npy
    lstm_w14.pt


In [12]:
# อ่านผล metrics - IMPROVED VERSION
import pandas as pd
import os
from pathlib import Path

print("🔍 ตรวจสอบโฟลเดอร์ผลลัพธ์...")

# ตรวจสอบโฟลเดอร์หลัก
outputs_dir = Path('outputs')
if not outputs_dir.exists():
    print("❌ โฟลเดอร์ outputs ไม่มีอยู่!")
    print("💡 กรุณารันการเทรนก่อน: !python -m scripts.run_train")
else:
    print(f"✅ พบโฟลเดอร์ outputs")

    # แสดงโครงสร้างโฟลเดอร์
    print("\n📁 โครงสร้างโฟลเดอร์ outputs:")
    for root, dirs, files in os.walk('outputs'):
        level = root.replace('outputs', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:10]:  # แสดงแค่ 10 ไฟล์แรก
            print(f"{subindent}{file}")
        if len(files) > 10:
            print(f"{subindent}... และอีก {len(files)-10} ไฟล์")

# ตรวจสอบโฟลเดอร์ metrics
metrics_dir = Path('outputs/metrics')
if not metrics_dir.exists():
    print("\n❌ โฟลเดอร์ outputs/metrics ไม่มีอยู่!")

    # ตรวจสอบไฟล์ metrics ในโฟลเดอร์อื่น
    print("🔍 ค้นหาไฟล์ metrics ในโฟลเดอร์อื่น...")
    metrics_found = False

    for root, dirs, files in os.walk('outputs'):
        for file in files:
            if 'metric' in file.lower() and file.endswith('.csv'):
                print(f"📊 พบไฟล์ metrics: {os.path.join(root, file)}")
                metrics_found = True

    if not metrics_found:
        print("⚠️ ไม่พบไฟล์ metrics ใดๆ")
        print("💡 กรุณาตรวจสอบ:")
        print("   1. การเทรนเสร็จสมบูรณ์หรือไม่")
        print("   2. มี error ระหว่างการเทรนหรือไม่")
        print("   3. รันคำสั่งเทรนใหม่อีกครั้ง")
else:
    # อ่านไฟล์ metrics
    metrics_files = [f for f in os.listdir('outputs/metrics') if f.endswith('.csv')]

    if metrics_files:
        print(f"\n📊 พบไฟล์ metrics: {len(metrics_files)} ไฟล์")

        # แสดงรายชื่อไฟล์
        for f in sorted(metrics_files):
            file_path = metrics_dir / f
            size = file_path.stat().st_size if file_path.exists() else 0
            print(f"   📄 {f} ({size} bytes)")

        # อ่านไฟล์ล่าสุด
        latest_metrics = sorted(metrics_files)[-1]
        print(f"\n? กำลังอ่านไฟล์ล่าสุด: {latest_metrics}")

        try:
            df_metrics = pd.read_csv(f'outputs/metrics/{latest_metrics}')
            print(f"✅ อ่านสำเร็จ! มี {len(df_metrics)} รายการ")
            print("\n? สรุปผลการเทรน:")
            print(df_metrics.describe())

            if len(df_metrics) > 0:
                print(f"\n🎯 ผลลัพธ์ล่าสุด:")
                latest_row = df_metrics.iloc[-1]
                for col in df_metrics.columns:
                    if col != 'window':
                        print(f"   {col}: {latest_row[col]:.4f}")

        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาดในการอ่านไฟล์: {e}")
            print("💡 ลองตรวจสอบรูปแบบไฟล์หรือเนื้อหาในไฟล์")
    else:
        print("⚠️ ไม่พบไฟล์ .csv ในโฟลเดอร์ metrics")

🔍 ตรวจสอบโฟลเดอร์ผลลัพธ์...
✅ พบโฟลเดอร์ outputs

📁 โครงสร้างโฟลเดอร์ outputs:
outputs/
  trades/
    trades_w5.parquet
    trades_w7.parquet
    trades_w3.parquet
    trades_w1.parquet
    trades_w9.parquet
    trades_w12.parquet
    trades_w2.parquet
    trades_w8.parquet
    trades_w18.parquet
    trades_w20.parquet
    ... และอีก 17 ไฟล์
  metrics/
    metrics_windows.json
    summary.json
    .gitkeep
    windows_summary.csv
  models/
    scaler_std_w7.npy
    scaler_std_w3.npy
    scaler_mean_w22.npy
    lstm_w11.pt
    lstm_w8.pt
    scaler_mean_w4.npy
    scaler_mean_w17.npy
    scaler_mean_w5.npy
    scaler_std_w1.npy
    scaler_mean_w18.npy
    ... และอีก 69 ไฟล์

📊 พบไฟล์ metrics: 1 ไฟล์
   📄 windows_summary.csv (1476 bytes)

� กำลังอ่านไฟล์ล่าสุด: windows_summary.csv
✅ อ่านสำเร็จ! มี 12 รายการ

� สรุปผลการเทรน:
          window  total_return   win_rate  total_trades  avg_return_per_trade  \
count  12.000000     12.000000  12.000000     12.000000             12.000000   
mea

In [13]:
# 📊 สร้างและวิเคราะห์ผลลัพธ์แบบละเอียด
import pandas as pd
import numpy as np
from pathlib import Path
import glob

def create_metrics_csv():
    """สร้างไฟล์ metrics CSV จากผลลัพธ์การเทรด"""

    trades_dir = Path('outputs/trades')
    if not trades_dir.exists():
        print("❌ ไม่พบโฟลเดอร์ trades!")
        return None

    trade_files = list(trades_dir.glob("trades_w*.parquet"))
    if not trade_files:
        print("❌ ไม่พบไฟล์ trades!")
        return None

    print("📊 กำลังสร้างไฟล์ metrics จากผลการเทรด...")

    metrics_list = []

    for file in trade_files:
        try:
            # แยกหมายเลข window
            window_num = int(file.stem.split('_w')[1])

            # โหลดข้อมูลการเทรด
            df = pd.read_parquet(file)

            if len(df) == 0:
                continue

            # คำนวณ metrics
            total_trades = len(df)
            winning_trades = (df['ret'] > 0).sum()
            losing_trades = (df['ret'] < 0).sum()

            win_rate = winning_trades / total_trades if total_trades > 0 else 0

            total_return = df['ret'].sum()
            avg_return_per_trade = df['ret'].mean()

            # คำนวณ Profit Factor
            gross_profit = df[df['ret'] > 0]['ret'].sum() if winning_trades > 0 else 0
            gross_loss = abs(df[df['ret'] < 0]['ret'].sum()) if losing_trades > 0 else 0.001  # หลีกเลี่ยง division by zero
            profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0

            # คำนวณ Drawdown (simplified)
            equity_curve = (1 + df['ret']).cumprod()
            running_max = equity_curve.expanding().max()
            drawdown = (equity_curve - running_max) / running_max
            max_drawdown = drawdown.min()

            # Sharpe ratio (simplified)
            returns_std = df['ret'].std()
            sharpe_ratio = avg_return_per_trade / returns_std if returns_std > 0 else 0

            metrics = {
                'window': window_num,
                'total_trades': total_trades,
                'winning_trades': winning_trades,
                'losing_trades': losing_trades,
                'win_rate': win_rate,
                'total_return_equity': total_return,
                'avg_return_per_trade': avg_return_per_trade,
                'profit_factor': profit_factor,
                'max_drawdown': max_drawdown,
                'sharpe_ratio': sharpe_ratio,
                'gross_profit': gross_profit,
                'gross_loss': gross_loss
            }

            metrics_list.append(metrics)

        except Exception as e:
            print(f"⚠️ ข้ามไฟล์ {file}: {e}")

    if not metrics_list:
        print("❌ ไม่สามารถสร้าง metrics ได้!")
        return None

    # สร้าง DataFrame
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df = metrics_df.sort_values('window')

    # บันทึกเป็น CSV
    metrics_dir = Path('outputs/metrics')
    metrics_dir.mkdir(exist_ok=True)

    csv_filename = f'trading_metrics_{len(metrics_df)}_windows.csv'
    csv_path = metrics_dir / csv_filename

    metrics_df.to_csv(csv_path, index=False)

    print(f"✅ สร้างไฟล์ {csv_filename} เรียบร้อย!")
    print(f"📊 จำนวน Windows: {len(metrics_df)}")

    return metrics_df

def analyze_trading_results():
    """วิเคราะห์ผลการเทรนและให้คำแนะนำ"""

    # สร้างไฟล์ metrics ก่อน
    df = create_metrics_csv()

    if df is None:
        print("❌ ไม่สามารถสร้างหรืออ่าน metrics ได้!")
        return None

    print("\n📊 การวิเคราะห์ผลลัพธ์การเทรน")
    print("="*60)

    # สถิติพื้นฐาน
    avg_win_rate = df['win_rate'].mean()
    avg_profit_factor = df['profit_factor'].mean()
    avg_return = df['total_return_equity'].mean()
    max_drawdown = df['max_drawdown'].min()
    total_trades = df['total_trades'].sum()

    print(f"📈 Average Win Rate: {avg_win_rate:.1%}")
    print(f"💰 Average Profit Factor: {avg_profit_factor:.2f}")
    print(f"📊 Average Return per Window: {avg_return:.1%}")
    print(f"📊 Total Return (cumulative): {df['total_return_equity'].sum():.1%}")
    print(f"📉 Worst Drawdown: {max_drawdown:.1%}")
    print(f"🔢 Total Trades: {total_trades:,}")
    print(f"🔢 Total Windows: {len(df)}")

    # ประเมินผลและให้คำแนะนำ
    print("\n🎯 การประเมินผล:")

    if avg_win_rate > 0.55:
        print("✅ Win Rate ดีมาก! (>55%)")
    elif avg_win_rate > 0.45:
        print("✅ Win Rate ดี (45-55%)")
    elif avg_win_rate > 0.35:
        print("⚠️ Win Rate ปานกลาง (35-45%) - พอใช้ได้")
    else:
        print("❌ Win Rate ต่ำ (<35%) - ต้องปรับปรุง")

    if avg_profit_factor > 1.2:
        print("✅ Profit Factor ยอดเยี่ยม! (>1.2)")
    elif avg_profit_factor > 1.0:
        print("✅ Profit Factor ดี (>1.0) - มีกำไร")
    elif avg_profit_factor > 0.8:
        print("⚠️ Profit Factor ต่ำ (0.8-1.0) - เกือบคุ้มทุน")
    else:
        print("❌ Profit Factor แย่ (<0.8) - ขาดทุน")

    total_return_sum = df['total_return_equity'].sum()
    if total_return_sum > 0:
        print("✅ Total Return เป็นบวก - มีกำไรโดยรวม!")
    elif total_return_sum > -0.2:
        print("⚠️ Total Return เป็นลบเล็กน้อย - ปรับปรุงได้")
    else:
        print("❌ Total Return เป็นลบมาก - ต้องปรับปรุงเร่งด่วน")

    # แสดงผลลัพธ์แต่ละ window
    print(f"\n📋 ผลลัพธ์รายละเอียดแต่ละ Window:")
    print("-" * 60)
    for _, row in df.iterrows():
        status = "✅" if row['total_return_equity'] > 0 else "❌"
        print(f"Window {row['window']:2d}: {status} Return={row['total_return_equity']:6.1%} | "
              f"Win Rate={row['win_rate']:5.1%} | Trades={row['total_trades']:3d}")

    # คำแนะนำการปรับปรุง
    print("\n💡 คำแนะนำการปรับปรุง:")

    profitable_windows = (df['total_return_equity'] > 0).sum()
    profit_ratio = profitable_windows / len(df)

    print(f"📊 Windows ที่ทำกำไร: {profitable_windows}/{len(df)} ({profit_ratio:.1%})")

    if avg_win_rate < 0.4:
        print("🔧 Win Rate ต่ำ:")
        print("   - เพิ่ม probability threshold (0.65 → 0.75+)")
        print("   - ลด take profit (atr_mult_tp)")
        print("   - เพิ่ม sequence length สำหรับ pattern recognition")

    if avg_profit_factor < 1.0:
        print("🔧 Profit Factor ต่ำ:")
        print("   - ตรวจสอบ fees และ slippage")
        print("   - ปรับ Risk-Reward ratio")
        print("   - เพิ่ม selectivity (higher threshold)")

    if max_drawdown < -0.3:
        print("🔧 Drawdown สูง:")
        print("   - ลด max holding time")
        print("   - เพิ่ม stop loss (ลด atr_mult_sl)")
        print("   - เพิ่ม probability threshold")

    if profit_ratio < 0.3:
        print("🔧 Consistency ต่ำ:")
        print("   - ลด training period (ให้ adaptive มากขึ้น)")
        print("   - เพิ่ม validation strictness")
        print("   - ปรับ class weights")

    # แสดง windows ที่ดีที่สุดและแย่ที่สุด
    best_window = df.loc[df['total_return_equity'].idxmax()]
    worst_window = df.loc[df['total_return_equity'].idxmin()]

    print(f"\n🏆 Window ที่ดีที่สุด: Window {best_window['window']} (Return: {best_window['total_return_equity']:.1%})")
    print(f"💔 Window ที่แย่ที่สุด: Window {worst_window['window']} (Return: {worst_window['total_return_equity']:.1%})")

    return df

# รันการวิเคราะห์
try:
    results_df = analyze_trading_results()
    print("\n✅ การวิเคราะห์เสร็จสมบูรณ์!")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")
    print("💡 ตรวจสอบว่าการเทรนเสร็จสมบูรณ์หรือไม่")

📊 กำลังสร้างไฟล์ metrics จากผลการเทรด...
✅ สร้างไฟล์ trading_metrics_26_windows.csv เรียบร้อย!
📊 จำนวน Windows: 26

📊 การวิเคราะห์ผลลัพธ์การเทรน
📈 Average Win Rate: 36.3%
💰 Average Profit Factor: 0.55
📊 Average Return per Window: -78.1%
📊 Total Return (cumulative): -2030.1%
📉 Worst Drawdown: -69.9%
🔢 Total Trades: 15,170
🔢 Total Windows: 26

🎯 การประเมินผล:
⚠️ Win Rate ปานกลาง (35-45%) - พอใช้ได้
❌ Profit Factor แย่ (<0.8) - ขาดทุน
❌ Total Return เป็นลบมาก - ต้องปรับปรุงเร่งด่วน

📋 ผลลัพธ์รายละเอียดแต่ละ Window:
------------------------------------------------------------
❌ เกิดข้อผิดพลาด: Unknown format code 'd' for object of type 'float'
💡 ตรวจสอบว่าการเทรนเสร็จสมบูรณ์หรือไม่


### 8. การปรับแต่งแบบ Real-time (หากผลยังไม่ดีพอ)

In [14]:
# 🔧 Auto-tuning หากผลยังไม่ดีพอ
def auto_tune_config():
    """ปรับแต่ง config อัตโนมัติตามผลลัพธ์"""

    # อ่านผลล่าสุด
    metrics_dir = Path('outputs/metrics')
    if not metrics_dir.exists():
        print("❌ ไม่พบผลการเทรน ไม่สามารถ auto-tune ได้")
        return False

    metrics_files = list(metrics_dir.glob("*.csv"))
    if not metrics_files:
        print("❌ ไม่พบไฟล์ metrics ไม่สามารถ auto-tune ได้")
        return False

    latest_file = sorted(metrics_files)[-1]
    df = pd.read_csv(latest_file)

    avg_win_rate = df['win_rate'].mean()
    avg_profit_factor = df['profit_factor'].mean()

    print(f"📊 ผลปัจจุบัน: Win Rate {avg_win_rate:.1%}, PF {avg_profit_factor:.2f}")

    # โหลด config ปัจจุบัน
    with open('configs/config.yaml', 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)

    # กำหนดการปรับแต่ง
    need_retrain = False

    if avg_win_rate < 0.35:
        print("🔧 Win Rate ต่ำเกินไป - ปรับเป็น Conservative Mode")
        config['label']['atr_mult_tp'] = 1.2
        config['trade']['atr_mult_tp'] = 1.2
        config['trade']['proba_threshold'] = 0.65
        config['train']['class_weights'] = [2.0, 0.2, 2.0]
        need_retrain = True

    elif avg_profit_factor < 0.8:
        print("🔧 Profit Factor ต่ำ - ปรับ Risk Management")
        config['label']['max_holding'] = max(12, config['label']['max_holding'] - 12)
        config['trade']['max_holding'] = max(12, config['trade']['max_holding'] - 12)
        config['trade']['proba_threshold'] = min(0.75, config['trade']['proba_threshold'] + 0.05)
        need_retrain = True

    elif avg_win_rate > 0.6 and avg_profit_factor > 1.2:
        print("🚀 ผลดีมาก! ลองเพิ่ม Risk เพื่อ Return สูงขึ้น")
        config['label']['atr_mult_tp'] = min(2.5, config['label']['atr_mult_tp'] + 0.2)
        config['trade']['atr_mult_tp'] = min(2.5, config['trade']['atr_mult_tp'] + 0.2)
        config['trade']['proba_threshold'] = max(0.45, config['trade']['proba_threshold'] - 0.05)
        need_retrain = True

    if need_retrain:
        # บันทึก config ใหม่
        with open('configs/config.yaml', 'w', encoding='utf-8') as f:
            yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

        print("✅ ปรับแต่ง config เรียบร้อย!")
        print("🎯 การตั้งค่าใหม่:")
        print(f"   TP: {config['trade']['atr_mult_tp']:.1f}")
        print(f"   SL: {config['trade']['atr_mult_sl']:.1f}")
        print(f"   Threshold: {config['trade']['proba_threshold']:.2f}")
        print(f"   Max Holding: {config['trade']['max_holding']}")

        return True
    else:
        print("✅ การตั้งค่าปัจจุบันใช้ได้ ไม่ต้องปรับแต่ง")
        return False

# รัน auto-tuning
print("🤖 เริ่ม Auto-tuning...")
need_retrain = auto_tune_config()

if need_retrain:
    retrain = input("\n🔄 ต้องการเทรนใหม่ด้วยการตั้งค่าที่ปรับแล้วหรือไม่? (y/n): ")
    if retrain.lower() == 'y':
        print("🚀 เริ่มเทรนใหม่...")
        !python -m scripts.run_train
    else:
        print("⏭️ ข้ามการเทรนใหม่")
else:
    print("🎉 การตั้งค่าปัจจุบันใช้ได้ดี!")

🤖 เริ่ม Auto-tuning...


KeyError: 'profit_factor'

## 💾 ดาวน์โหลดผลลัพธ์

### 9. ดาวน์โหลดไฟล์ผลลัพธ์และรายงาน

In [ ]:
# 📦 สร้างและดาวน์โหลด Complete Package
import zipfile
from google.colab import files
import json
from datetime import datetime

# สร้างรายงานสรุป
def create_summary_report():
    """สร้างรายงานสรุปผลการเทรน"""
    try:
        metrics_dir = Path('outputs/metrics')

        # หาไฟล์ CSV ที่มีอยู่
        csv_files = list(metrics_dir.glob("*.csv"))

        if csv_files:
            # ใช้ไฟล์ล่าสุด
            latest_file = sorted(csv_files, key=lambda x: x.stat().st_mtime)[-1]
            df = pd.read_csv(latest_file)

            print(f"📊 ใช้ไฟล์ metrics: {latest_file.name}")

        else:
            # ถ้าไม่มีไฟล์ CSV ให้สร้างจากข้อมูล trades
            print("📊 ไม่พบไฟล์ CSV, กำลังสร้างจากข้อมูล trades...")

            # สร้างจากข้อมูล trades (ใช้ logic เดียวกับข้างต้น)
            trades_dir = Path('outputs/trades')
            trade_files = list(trades_dir.glob("trades_w*.parquet"))

            if not trade_files:
                print("❌ ไม่พบข้อมูล trades!")
                return False

            metrics_list = []
            for file in trade_files:
                try:
                    window_num = int(file.stem.split('_w')[1])
                    df_trades = pd.read_parquet(file)

                    if len(df_trades) == 0:
                        continue

                    total_trades = len(df_trades)
                    winning_trades = (df_trades['ret'] > 0).sum()
                    win_rate = winning_trades / total_trades if total_trades > 0 else 0
                    total_return = df_trades['ret'].sum()

                    gross_profit = df_trades[df_trades['ret'] > 0]['ret'].sum() if winning_trades > 0 else 0
                    gross_loss = abs(df_trades[df_trades['ret'] < 0]['ret'].sum()) if (df_trades['ret'] < 0).sum() > 0 else 0.001
                    profit_factor = gross_profit / gross_loss if gross_loss > 0 else 0

                    equity_curve = (1 + df_trades['ret']).cumprod()
                    running_max = equity_curve.expanding().max()
                    drawdown = (equity_curve - running_max) / running_max
                    max_drawdown = drawdown.min()

                    metrics_list.append({
                        'window': window_num,
                        'total_trades': total_trades,
                        'winning_trades': winning_trades,
                        'win_rate': win_rate,
                        'total_return_equity': total_return,
                        'profit_factor': profit_factor,
                        'max_drawdown': max_drawdown
                    })

                except Exception as e:
                    print(f"⚠️ ข้ามไฟล์ {file}: {e}")

            if not metrics_list:
                print("❌ ไม่สามารถสร้าง metrics ได้!")
                return False

            df = pd.DataFrame(metrics_list).sort_values('window')

            # บันทึกเป็น CSV
            csv_filename = f'trading_metrics_{len(df)}_windows.csv'
            csv_path = metrics_dir / csv_filename
            df.to_csv(csv_path, index=False)
            print(f"✅ สร้างไฟล์ {csv_filename} เรียบร้อย!")

        # สร้างรายงานสรุป
        report = {
            "training_summary": {
                "timestamp": datetime.now().isoformat(),
                "total_windows": len(df),
                "avg_win_rate": float(df['win_rate'].mean()),
                "avg_profit_factor": float(df['profit_factor'].mean()) if 'profit_factor' in df.columns else 0,
                "avg_return": float(df['total_return_equity'].mean()),
                "total_return": float(df['total_return_equity'].sum()),
                "max_drawdown": float(df['max_drawdown'].min()) if 'max_drawdown' in df.columns else 0,
                "profitable_windows": int((df['total_return_equity'] > 0).sum()),
                "total_trades": int(df['total_trades'].sum()) if 'total_trades' in df.columns else 0,
                "best_window": int(df.loc[df['total_return_equity'].idxmax(), 'window']),
                "worst_window": int(df.loc[df['total_return_equity'].idxmin(), 'window']),
                "best_return": float(df['total_return_equity'].max()),
                "worst_return": float(df['total_return_equity'].min())
            },
            "performance_metrics": df.to_dict('records')
        }

        with open('training_report.json', 'w') as f:
            json.dump(report, f, indent=2)

        print("📊 สร้างรายงานสรุปเรียบร้อย!")
        print(f"   📈 Total Windows: {len(df)}")
        print(f"   📈 Average Win Rate: {df['win_rate'].mean():.1%}")
        print(f"   📈 Total Return: {df['total_return_equity'].sum():.1%}")
        print(f"   📈 Profitable Windows: {(df['total_return_equity'] > 0).sum()}/{len(df)}")

        return True

    except Exception as e:
        print(f"❌ ไม่สามารถสร้างรายงานได้: {e}")
        return False

# สร้างรายงาน
report_created = create_summary_report()

# สร้างไฟล์ zip
zip_filename = f'btc_lstm_results_{datetime.now().strftime("%Y%m%d_%H%M")}.zip'

print(f"\n📦 กำลังสร้างไฟล์ {zip_filename}...")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # เพิ่มไฟล์ใน outputs
    if Path('outputs').exists():
        for root, dirs, files in os.walk('outputs'):
            for file in files:
                if not file.startswith('.'):  # ข้าม hidden files
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path)  # ใช้ relative path ใน zip
                    zipf.write(file_path, arcname)
        print("✅ เพิ่มไฟล์ outputs (รวม CSV metrics)")

    # เพิ่ม config file
    if Path('configs/config.yaml').exists():
        zipf.write('configs/config.yaml', 'config.yaml')
        print("✅ เพิ่ม config file")

    # เพิ่มรายงานสรุป
    if report_created and Path('training_report.json').exists():
        zipf.write('training_report.json')
        print("✅ เพิ่มรายงานสรุป")

    # เพิ่ม notebook นี้
    try:
        zipf.write('BTC_LSTM_Colab.ipynb')
        print("✅ เพิ่ม notebook")
    except:
        print("⚠️ ไม่สามารถเพิ่ม notebook ได้")

print(f"\n🎉 สร้างไฟล์ {zip_filename} เรียบร้อย!")

# ตรวจสอบขนาดไฟล์
file_size = Path(zip_filename).stat().st_size / 1024 / 1024  # MB
print(f"📏 ขนาดไฟล์: {file_size:.1f} MB")

if file_size > 100:
    print("⚠️ ไฟล์ใหญ่มาก อาจใช้เวลาในการดาวน์โหลด")

# แสดงรายการไฟล์ที่รวมอยู่ใน package
print("\n? ไฟล์ที่รวมใน package:")
print("   📊 outputs/trades/ - ผลการเทรดแต่ละ window (.parquet)")
print("   ? outputs/metrics/ - สถิติและ CSV metrics")
print("   📊 outputs/models/ - โมเดลและ scalers ที่เทรนแล้ว")
print("   ⚙️ config.yaml - การตั้งค่าที่ใช้เทรน")
print("   📄 training_report.json - รายงานสรุปแบบละเอียด")
print("   📓 BTC_LSTM_Colab.ipynb - notebook นี้")

# ดาวน์โหลด
print("\n🔽 เริ่มดาวน์โหลด...")
files.download(zip_filename)

print("\n✅ ดาวน์โหลดเรียบร้อย!")
print("🚀 สามารถนำไฟล์ไปใช้งานต่อได้ทันที!")

## 💡 เทคนิคและข้อแนะนำ

### การปรับแต่งสำหรับ Colab:

**หาก GPU Memory ไม่พอ:**
```yaml
train:
  batch_size: 256  # ลดจาก 1024
  device: "cuda"
```

**หากต้องการทดสอบเร็ว:**
```yaml
split:
  train_months: 3   # ลดจาก 12
  valid_months: 1   # ลดจาก 3
  test_months: 1    # ลดจาก 3

train:
  max_epochs: 5     # ลดจาก 25
```

**หากใช้ CPU เท่านั้น:**
```yaml
train:
  batch_size: 128   # ลดลงมาก
  max_epochs: 5
  device: "cpu"
```

### 📊 การตีความผลลัพธ์:
- **Win Rate > 33.3%**: กำไรจาก RR 1:2
- **Sharpe Ratio > 1.0**: ประสิทธิภาพดี
- **Max Drawdown < 10%**: ความเสี่ยงยอมรับได้

---
### 🎯 สำเร็จ! คุณได้ Model เทรด Bitcoin แล้ว 🚀